In [ ]:
from os import path

import astropy.units as u
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# Custom
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic

In [ ]:
mw = gp.BovyMWPotential2014()

Load default bar model coefficients:

In [ ]:
S = np.load('../data/Sn9l19m.npy')
Serr = np.load('../data/Sn9l19m_err.npy')

In [ ]:
plt.plot(np.abs(S[:, 0, 0]))
plt.plot(np.abs(Serr[:, 0, 0]))
plt.yscale('log')

In [ ]:
plt.plot(np.abs(S[2, :, 0]))
plt.plot(np.abs(Serr[2, :, 0]))
plt.yscale('log')

In [ ]:
plt.plot(np.abs(S[2, 8, :]))
plt.plot(np.abs(Serr[2, 8, :]))
plt.yscale('log')

---

### Like Banik and Bovy, add bar model to MWPotential2014

Here we just set up the default potential model, before scaling the bar size

In [ ]:
pot = gp.CCompositePotential()
pot['disk'] = mw['disk']
pot['halo'] = mw['halo']
pot['bar'] = gp.SCFPotential(m=5e9 / 9, r_s=1., # MAGIC NUMBER: just believe
                             Snlm=S,
                             units=galactic)

In [ ]:
xyz = np.zeros((3, 1024)) + 1e-8
xyz[0] = np.linspace(0, 30, xyz.shape[1])
    
plt.figure(figsize=(8, 5))
plt.plot(xyz[0], mw.circular_velocity(xyz).to_value(u.km/u.s))
plt.plot(xyz[0], pot.circular_velocity(xyz).to_value(u.km/u.s))
plt.xlim(0, 30)
plt.ylim(0, 300)

---

## Visualize surface density

In [ ]:
diskbar = gp.CCompositePotential()
diskbar['disk'] = mw['disk']
diskbar['bar'] = pot['bar']

In [ ]:
grid = np.linspace(-15, 15, 64)
x,y,z = np.meshgrid(grid, grid, grid)
xyz = np.stack((x, y, z))
dx = grid[1] - grid[0]

In [ ]:
dens = diskbar.density(xyz.reshape(3, -1))

In [ ]:
surf_dens = np.sum(dens.reshape(xyz.shape[1:]), axis=-1) * dx * u.kpc

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.pcolormesh(xyz[0, ..., -1], xyz[1, ..., -1], surf_dens.T.value, 
              norm=mpl.colors.LogNorm())

---

## Figure out bar scaling with pattern speed

In [ ]:
disknobar = gp.CCompositePotential()
disknobar['disk'] = mw['disk']
disknobar['bulge'] = mw['bulge']

In [ ]:
diskbar1 = gp.CCompositePotential()
diskbar1['disk'] = mw['disk']
diskbar1['bar'] = gp.SCFPotential(m=5e9 / 10., r_s=1., # MAGIC NUMBER: just believe
                                 Snlm=S,
                                 units=galactic)

diskbar2 = gp.CCompositePotential()
diskbar2['disk'] = mw['disk']
diskbar2['bar'] = gp.SCFPotential(m=5e9 / 10., r_s=2, # MAGIC NUMBER: just believe
                                 Snlm=S,
                                 units=galactic)

In [ ]:
grid = np.linspace(-10, 10, 32)
x,y,z = np.meshgrid(grid, grid, grid)
xyz = np.stack((x, y, z))
dx = grid[1] - grid[0]

In [ ]:
dens = diskbar1.density(xyz.reshape(3, -1))
surf_dens1 = np.sum(dens.reshape(xyz.shape[1:]), axis=-1) * dx * u.kpc

dens = diskbar2.density(xyz.reshape(3, -1))
surf_dens2 = np.sum(dens.reshape(xyz.shape[1:]), axis=-1) * dx * u.kpc

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].pcolormesh(xyz[0, ..., -1], xyz[1, ..., -1], surf_dens1.T.value, 
              norm=mpl.colors.LogNorm())
axes[1].pcolormesh(xyz[0, ..., -1], xyz[1, ..., -1], surf_dens2.T.value, 
              norm=mpl.colors.LogNorm())

In [ ]:
xyz = np.zeros((3, 1024)) + 1e-8
xyz[0] = np.linspace(0, 30, xyz.shape[1])
    
plt.figure(figsize=(8, 5))
plt.plot(xyz[0], disknobar.circular_velocity(xyz).to_value(u.km/u.s))
plt.plot(xyz[0], diskbar1.circular_velocity(xyz).to_value(u.km/u.s))
plt.plot(xyz[0], diskbar2.circular_velocity(xyz).to_value(u.km/u.s))
plt.xlim(0, 30)
plt.ylim(0, 300)

In [ ]:
solar_R = [8.1, 0, 0] * u.kpc
print(disknobar.circular_velocity(solar_R),
      diskbar1.circular_velocity(solar_R),
      diskbar2.circular_velocity(solar_R))

---

In [ ]:
from scipy.optimize import minimize

In [ ]:
def corot_func(r_cr, Omega):
    vc = disknobar.circular_velocity([r_cr, 0., 0.])
    return abs(vc - Omega*r_cr*u.kpc).decompose().value[0]

In [ ]:
def get_bar_model(Omega):
    res = minimize(corot_func, x0=4., args=(Omega, ))
    r_cr = res.x[0]
    r_s = r_cr / 3.2 # 3.2 scales this to the value WZ2012 use (60 km/s/kpc)
    
    return gp.SCFPotential(m=5e9 / 10., r_s=r_s, # 10 is a MAGIC NUMBER: just believe
                           Snlm=S,
                           units=galactic)